In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime

import time

import pandas as pd

c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
c_driver = webdriver.Chrome(options=c_options)

In [2]:
# URL base
url = "https://www.tesla.com/es_ES"
params = 'arrangeby=plh&zip=28020&range=0'

# Configuración del diccionario de modelos
cars_types = {
    'Model S': 'ms',
    'Model 3': 'm3',
    'Model X': 'mx',
    'Model Y': 'my'
}


types = {
    'Nuevo': 'new',
    'Ocasion': 'used'
}


In [5]:
# Diccionario para almacenar los datos
data = []
for type in types:
    for car_type in cars_types:
        current_car = cars_types[car_type]
        current_type = types[type]
        # Navegar a la página del inventario del modelo específico
        c_driver.get(f'{url}/inventory/{current_type}/{current_car}?{params}')
        time.sleep(10)  # Esperar para que cargue la página

        # Buscar los coches en la página
        cars = c_driver.find_elements(By.XPATH, '/html/body/div[1]/div/div[1]/main/div/article')
        for car in cars:
            
            traction = car.find_element(By.XPATH, './section[1]/div[1]/div[1]').text
            
            if current_type == 'new':
                price = car.find_element(By.XPATH, './section[1]/div[2]/div[1]/div/span[1]').text
                year = datetime.now().year
            else:
                price = car.find_element(By.XPATH, './section[1]/div[2]/div/div/span').text
                year = car.find_element(By.XPATH, './section[1]/div[1]/h3/span[1]').text
            
            try:
                km = car.find_element(By.XPATH, './section[1]/div[1]/div[2]/span').text or 0
            except NoSuchElementException:
                km = 0

            range_ = car.find_element(By.XPATH, './section[5]/div/ul/li[1]/div[1]/span[1]').text
            max_vel = car.find_element(By.XPATH, './section[5]/div/ul/li[2]/div[1]/span[1]').text
            zero_hundred = car.find_element(By.XPATH, './section[5]/div/ul/li[3]/div[1]/span[1]').text

            # Agregar los datos al diccionario
            data.append({
                'sell_type': type,
                'car_type': car_type,
                'year': year,
                'traction': traction,
                'price': price,
                'km': km,
                'range': range_,
                'max_vel': max_vel,
                'zero_hundred': zero_hundred
            })

In [7]:
df = pd.DataFrame(data)
df

,sell_type,car_type,year,traction,price,km,range,max_vel,zero_hundred
0,Nuevo,Model S,2024,"Model S, tracción a las cuatro ruedas",89.100 €,Cuentakilómetros con 6196 km,634,250,"3,2"
1,Nuevo,Model S,2024,"Model S, tracción a las cuatro ruedas",99.600 €,Cuentakilómetros con 503 km,634,250,"3,2"
2,Nuevo,Model S,2024,Plaid,101.700 €,Cuentakilómetros con 3652 km,600,322,"2,1"
3,Nuevo,Model S,2024,"Model S, tracción a las cuatro ruedas",102.270 €,Cuentakilómetros con 99 km,634,250,"3,2"
4,Nuevo,Model S,2024,"Model S, tracción a las cuatro ruedas",102.270 €,0,634,250,"3,2"
...,...,...,...,...,...,...,...,...,...
181,Ocasion,Model X,2023,Plaid,85.500 €,Cuentakilómetros con 27.271 km,543,262,"2,6"
182,Ocasion,Model X,2023,Plaid,86.500 €,Cuentakilómetros con 21.471 km,543,262,"2,6"
183,Ocasion,Model X,2023,Plaid,87.600 €,Cuentakilómetros con 20.074 km,543,262,"2,6"
184,Ocasion,Model Y,2024,Model Y Tracción trasera,40.300 €,Cuentakilómetros con 734 km,455,217,"6,9"


In [8]:
df.isna().sum()

sell_type       0
car_type        0
year            0
traction        0
price           0
km              0
range           0
max_vel         0
zero_hundred    0
dtype: int64

In [9]:
df.to_csv('tesla.com.csv')

In [10]:
# 06 / 12 / 2024